# Generic Template for a Basic Encrypted Order Book (via Sorted Arrays)

An *encrypted order book* is an [order book](https://en.wikipedia.org/wiki/Order_book) in which orders are submitted in encrypted form and remain encrypted until they are revealed as part of a matching pair. This document presents a generic template for an encrypted order book that relies on maintaining sorted arrays of submitted orders. An encrypted order book implementation can be built based on this template by providing implementations of the base functions (for example, using a secure multi-party computation or homomorphic encryption scheme) that can operate on encrypted data.

## Data Structures and Functions

The data structures and functions below are reference implementations that represent and operate on data in the clear. In order to implement an encrypted order book, only the [**base functions**](#base-functions-on-orders) must be implemented in such a way that they can be applied to encrypted orders.

### Order Data Structure<a id='order-data-structure'></a>

An order consists of an integer price and an integer quantity. The particular commodity, security, or other category of good to which the order corresponds is assumed to be tracked in some manner that falls outside the scope of the template presented in this article.

In [17]:
from __future__ import annotations

class order:
    """
    Individual orders (consisting of an integer price and an integer quantity).
    """
    def __init__(self, price: int, quantity: int):
        self.price: int = price
        self.quantity: int = quantity
    
    def __repr__(self: order) -> str:
        """
        Return a human-readable and ``eval``-friendly representation.
        """
        return 'order' + str((self.price, self.quantity))

### Primitive Operations for Simple Types<a id='primitive-operations-for-simple-types'></a>

To implement an encrypted order book, every [**base function**](#base-functions-on-orders) must be implemented using a secure computation scheme. The base functions can be implemented either (1) via composition of primitive operations that can be applied to encrypted integer and boolean values or (2) as high-level, black-box functions (*e.g.*, custom-built MPC protocols) that be applied to encrypted orders. To accommodate scenario (1), this section provides a comprehensive collection of primitive operations (in the form of reference implementations that operate on plaintext integer and boolean values). For completeness, these reference implementations are also used within the reference implementations of the [**base functions**](#base-functions-on-orders) (both directly and via [**primitive operations for non-simple types**](#primitive-operations-for-non-simple-types)).

In [18]:
def or_(a: bool, b: bool) -> bool:
    """
    Returns the disjunction of the two boolean arguments.
    """
    return a or b

def if_then_else(c: bool, n: int, m: int) -> int:
    """
    Ternary operator that returns the second argument if the first is true,
    and otherwise returns the third argument.
    """
    return n if c else m

def equal_to_zero(n: int) -> bool:
    """
    Returns a boolean that represents whether the argument is equal to zero.
    """
    return n == 0

def less_than(n: int, m: int) -> bool:
    """
    Returns a boolean value that represents whether the first argument is
    strictly less than the second argument. 
    """
    return n < m

def minus(n: int, m: int) -> int:
    """
    Return the integer result of subtracting the second argument
    from the first argument.
    """
    return n - m

def minimum(n: int, m: int) -> int:
    """
    Return the smaller of the two arguments.
    """
    return if_then_else(less_than(n, m), n, m)

def maximum(n: int, m: int) -> int:
    """
    Return the larger of the two arguments.
    """
    return if_then_else(less_than(n, m), m, n)

### Primitive Operations for Non-Simple Types<a id='primitive-operations-for-non-simple-types'></a>

It is natural to define variants of [**primitive operations defined on simple types**](#primitive-operations-for-simple-types) for more complex types (such as tuples) and/or to user-defined types (such as [**orders**](#order-data-structure)). The primitive operations below are defined for instances of the [**order data structure**](#order-data-structure) and pairs thereof (using only [**primitive operations defined on simple types**](#primitive-operations-for-simple-types)). In a more mature language and compiler framework, polymorphic primitive operations corresponding to those below might be made available.

In [19]:
def if_then_else_order(c: bool, x: order, y: order) -> order:
    """
    Ternary operator that returns the second argument if the first is true,
    and otherwise returns the third argument.
    """
    price = if_then_else(c, x.price, y.price)
    quantity = if_then_else(c, x.quantity, y.quantity)
    return order(price, quantity)

def minimum_order(x: order, y: order) -> order:
    """
    Return the order with the smaller price.
    """
    condition = less_than(x.price, y.price)
    price = if_then_else(condition, x.price, y.price)
    quantity = if_then_else(condition, x.quantity, y.quantity)
    return order(price, quantity)

def maximum_order(x: order, y: order) -> order:
    """
    Return the order with the larger price.
    """
    condition = less_than(x.price, y.price)
    price = if_then_else(condition, y.price, x.price)
    quantity = if_then_else(condition, y.quantity, x.quantity)
    return order(price, quantity)

def if_then_else_pair(
        c: bool,
        p: tuple[order, order],
        q: tuple[order, order]
    ) -> tuple[order, order]:
    """
    Ternary operator that returns the second argument if the first is true,
    and otherwise returns the third argument.
    """
    return (
        if_then_else(c, p[0], q[0]),
        if_then_else(c, p[1], q[1])
    )

### Base Functions on Orders<a id='base-functions-on-orders'></a>

To implement an encrypted order book, every function in this section must be implemented using a secure computation scheme (such that the function can be applied to pairs of encrypted orders). This can either be accomplished by (1) leveraging primitive operations on [**simple**](#primitive-operations-for-simple-types) and [**non-simple**](#primitive-operations-for-non-simple-types) types that can be applied to encrypted values or (2) implementing the functions directly as black boxes using some other approach.

Orders can be sorted according to price, except that orders that have a ``quantity`` of ``0`` should always appear earlier. In order to implement an encrypted order book, it must be possible to evaluate the ``ascending`` and ``descending`` functions presented below on pairs of encrypted orders. These functions are used to maintain [**encrypted sorted arrays**](#sorted-array-data-structure) of orders.

In [20]:
from typing import Callable

oo: float = float('inf')

def descending(pair: tuple[order, order]) -> tuple[order, order]:
    """
    Return a tuple in which the two orders in the supplied tuple are
    arranged in descending order by price, except that any order with
    a quantity of zero appears earlier. Effectively, the below algorithm
    is implemented using primitive operations:

        (x, y) = pair

        if x.quantity == 0:
            return (x, y)

        if y.quantity == 0:
            return (y, x)

        return (max(x, y), min(x, y))

    """
    (x, y) = pair

    x_quantity_is_zero = equal_to_zero(x.quantity)
    y_quantity_is_zero = equal_to_zero(y.quantity)

    return if_then_else_pair(
        y_quantity_is_zero,
        (y, x),
        if_then_else_pair(
            x_quantity_is_zero,
            (x, y),
            (maximum_order(x, y), minimum_order(x, y))
        )
    )

def ascending(pair: tuple[order, order]) -> tuple[order, order]:
    """
    Return a tuple in which the two orders in the supplied tuple are
    arranged in ascending order by price, except that any order with
    a quantity of zero appears earlier.
    """
    (x, y) = pair

    x_quantity_is_zero = equal_to_zero(x.quantity)
    y_quantity_is_zero = equal_to_zero(y.quantity)

    return if_then_else_pair(
        y_quantity_is_zero,
        (y, x),
        if_then_else_pair(
            x_quantity_is_zero,
            (x, y),
            (minimum_order(x, y), maximum_order(x, y))
        )
    )

In order to implement an encrypted order book, it must be possible to evaluate the ``matched`` and ``updated`` functions defined below on pairs of encrypted orders. These are used by the [**order book implementation**](#order-book-implementation) to determine whether a match has occurred, and to update the order book's arrays in a way that reflects the effects of an executed match.

In [21]:
from typing import Optional

def matched(ask_bid: tuple[order, order]) -> Optional[tuple[int, int]]:
    """
    Return a tuple indicating the price and quantity of a transaction
    (if one is possible) or ``None`` (if one is not possible).
    """
    (ask, bid) = ask_bid
    quantity = minimum(ask.quantity, bid.quantity)
    bid_lower_than_ask = less_than(bid.price, ask.price)
    return_none = or_(bid_lower_than_ask, equal_to_zero(quantity))

    # The value of ``return_none`` is decrypted and the computation below
    # returns a decrypted price and quantity if ``return_none == False``.
    return (
        None
        if return_none else
        (ask.price, quantity)
    )

def updated(ask_bid: tuple[order, order]) -> tuple[order, order]:
    """
    Return a tuple that either (1) is identical to the input tuple because
    there is no match or (2) updated to reflect the transaction.
    """
    (ask, bid) = ask_bid
    quantity = minimum(ask.quantity, bid.quantity)
    return if_then_else_pair(
        less_than(bid.price, ask.price),
        ask_bid, # No change.
        ( # Perform transaction and return updated orders.
            order(ask.price, minus(ask.quantity, quantity)),
            order(bid.price, minus(bid.quantity, quantity))
        )
    )

### Sorted Array Data Structure<a id="sorted-array-data-structure"></a>

A sorted array maintains orders in either an ascending or a descending arrangement according to a specific sorting function (that operates on one pair of orders at a time).

In [22]:
from typing import Callable

class array(list):
    """
    Arrays of orders (usually maintained in ascending or descending order).
    """
    def arrange(
            self: array,
            function: Callable[[tuple[order, order]], tuple[order, order]]
        ):
        """
        Arrange this instance such that every pair is sorted
        according to the supplied function.
        """
        for i in range(len(self) - 1):
            pair = (self[i], self[i + 1])
            (x, y) = function(pair)
            self[i] = x
            self[i + 1] = y

    def add(
            self: array,
            entry: order,
            function: Callable[[tuple[order, order]], tuple[order, order]]
        ):
        """
        Add a new entry and arrange the array.
        """
        self[0] = entry
        self.arrange(function)

The example below tests the ``array`` data structure.

In [23]:
from random import randint

oo: float = float('inf')

length = 3
rs = [order(randint(1, 6), 1) for _ in range(length)]
bids = array([order(0, 0) for _ in range(length)])
for r in rs:
    bids.add(r, ascending)

rs = [order(randint(4, 9), 1) for _ in range(length)]
asks = array([order(oo, 0) for _ in range(length)])
for r in rs:
    asks.add(r, descending)

print(asks, bids)

[order(8, 1), order(7, 1), order(4, 1)] [order(1, 1), order(1, 1), order(6, 1)]


## Order Book Implementation<a id='order-book-implementation'></a>

The order book implementation maintains two sorted arrays (one for asks and one for bids). When a new order is added, the below steps are executed.
1. The order is added to the appropriate array and the array is arranged.
2. The last entries from the two arrays are processed using the matching functions.
3. If a match occurred, the arrays are arranged again and execution returns to step (2) above (in case there are new matches given the rearranged arrays). 

In [24]:
from typing import Optional

class book:
    """
    An order book that allows the submission of bid and ask orders,
    as well as matching of orders.
    """
    def __init__(self: book, size: int):
        """
        Initialize an order book of the specified size. No more
        bid orders can be submitted after ``size`` bid orders have
        been submitted (and likewise for ask orders).
        """
        self.asks = array([order(oo, 0) for _ in range(size)])
        self.bids = array([order(0, 0) for _ in range(size)])

    def ask(self: book, order: order):
        """
        Add an ask order (and sort the array of asks).
        """
        self.asks.add(order, descending)

    def bid(self: book, order: order):
        """
        Add a bid order (and sort the array of bids).
        """
        self.bids.add(order, ascending)

    def match(self: book) -> Optional[tuple[int, int]]:
        """
        Update the last entries of the ask and bid arrays using
        the matching function. Then, sort both arrays to move any
        exhausted orders to earlier positions.
        """
        ask = self.asks[-1]
        bid = self.bids[-1]
        outcome = matched((ask, bid))
        
        if outcome is not None:
            (ask, bid) = updated((ask, bid))
            self.asks[-1] = ask
            self.bids[-1] = bid
            self.asks.arrange(descending)
            self.bids.arrange(ascending)
            
        return outcome

The example below tests the ``book`` data structure on a sequence of orders.

In [25]:
from random import randint, choice, seed

seed(1)
b = book(6)
orders = [
    (
        order(randint(1, 9) * 100, randint(1, 9)),
        choice(['ask', 'bid'])
    )
    for _ in range(7)
]
for (o, ask_or_bid) in orders:
    print(ask_or_bid, o)
    getattr(b, ask_or_bid)(o)
    transaction = ()
    while transaction is not None:
        transaction = b.match()
        print(transaction)
    print()

bid order(300, 2)
None

bid order(200, 8)
None

ask order(800, 7)
None

ask order(200, 8)
(200, 2)
(200, 6)
None

ask order(700, 7)
None

ask order(800, 5)
None

ask order(200, 6)
(200, 2)
None



In [10]:
# End of file.